# Plan your meals from CSA shit!
Recipe data source: Epicurious.com

In [ ]:
## Import all libraries
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from bs4 import BeautifulSoup
import numpy as np
from collections import defaultdict
import urllib2, time, pickle

# SQL packages
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
from nltk.stem.wordnet import WordNetLemmatizer

### I am scraping for the recipes using vegetables typically grown in my old CSA, because they have that list and also because they have some sample boxes that I can test my algorithm on!

In [ ]:
testCsaUrl = "http://driftlessorganics.com/csa-2/recipes/"
page = urllib2.urlopen(testCsaUrl)
vegetableSoup = BeautifulSoup(page, "html.parser")
veggiesHTML = vegetableSoup.findAll('span', style="color: #339966;")

vegetables = []
for veggie in veggiesHTML:
    if veggie.a:
        vegetables.append(veggie.a.get_text().lower())
 
### Great I have a list of vegetables! Now to clean them up! Like split up Cauliflower and Romanesco!
vegetablesSplit = [vegetable.split('&',1) for vegetable in vegetables]
veggieList = sum(vegetablesSplit, [])
#veggiesList = veggieList[:15]

In [ ]:
gm


foodSpoilURL = "http://ohmyveggies.com/how-to-prevent-food-spoilage-with-careful-meal-planning/"
#page = urllib2.urlopen(foodSpoilURL)
hdr = {'User-Agent':'Mozilla/5.0'}
req = urllib2.Request(foodSpoilURL,headers=hdr)
page = urllib2.urlopen(req)
foodSpoilSoup = BeautifulSoup(page, "html.parser")
foodSpoilHtml = foodSpoilSoup.findAll("p")

foodSpoilsinDate12Days = foodSpoilHtml[7].get_text().split()
foodSpoilsin35Days = foodSpoilHtml[9].get_text().split()
foodSpoilsin67Days = foodSpoilHtml[11].get_text().split()
foodSpoilsGreatWeek = foodSpoilHtml[13].get_text().split()

#foodSpoilDates = [foodSpoilsinDate12Days, foodSpoilsin35Days, foodSpoilsin67Days, foodSpoilsGreatWeek]

foodSpoilsinDate12Days_lemma = [lemmatizePhrase(str(veggie).lower()) for veggie in foodSpoilsinDate12Days]   
foodSpoilsinDate35Days_lemma = [lemmatizePhrase(str(veggie).lower()) for veggie in foodSpoilsin35Days]  
foodSpoilsinDate67Days_lemma = [lemmatizePhrase(str(veggie).lower()) for veggie in foodSpoilsin67Days] 
foodSpoilsGreatWeek_lemma = [lemmatizePhrase(str(veggie).lower()) for veggie in foodSpoilsGreatWeek] 
foodSpoilsGreatWeek_lemma = foodSpoilsGreatWeek_lemma[:13] + foodSpoilsGreatWeek_lemma[22:]
foodSpoilsGreatWeek_lemma.append('dill')
foodSpoilsinDate35Days_lemma = foodSpoilsinDate35Days_lemma[:2]+foodSpoilsinDate35Days_lemma[14:]
foodSpoilsinDate35Days_lemma.append('basil')

veggies1 = foodSpoilsinDate12Days_lemma + foodSpoilsinDate35Days_lemma + \
foodSpoilsinDate67Days_lemma + foodSpoilsGreatWeek_lemma
veggies1.remove('and')
veggies1.remove('other')
veggies1.remove('hot')
veggies2 = list(set(veggies1)-set(veggieList))

In [ ]:
list(set(veggieList)-set(veggies1))

### Scrap epicurious.com for recipes for every vegetable in my CSA list!
because they are free, I see they have tags (gluten-free/kid friendly etc), reviews, ratings and sort recipes by ingredients, calorie info

*** Will not scrap data from a recipe without images

In [ ]:
browser = webdriver.Chrome("chromedriver")
def getRecipes(df, veggiesList, pageNumber, visiting = None):
    if visiting is None:
        visiting = set()
        if len(veggiesList) == 0:
            return df
        else:
            veggie = veggiesList.pop()
            visiting.add(veggie)
            print veggie, pageNumber
        
    recipes = defaultdict(list)
    
    pageSize = 30
    resultOffset = pageNumber*pageSize + 1
    url = \
    "http://www.epicurious.com/tools/searchresults?search=%s&sort=1&type=simple&pageNumber=%s&pageSize=%s&resultOffset=%s"%\
    (list(visiting)[0], pageNumber, pageSize, resultOffset)
    browser.get(url)
    time.sleep(2.0)
    recipeListPage = browser.page_source
    soup2 = BeautifulSoup(recipeListPage, "html.parser")
    
    
    #allRecipes.append(soup2.find('div', class_='sr_rows clearfix firstResult').a.get('href'))
    if soup2.find('div', class_='sr_rows clearfix firstResult') is None:
        return getRecipes(df, veggiesList, 1, visiting = None)
        
    recipes['recipe'].append(soup2.find('div', class_='sr_rows clearfix firstResult').a.get('href'))
    recipes['searchIngredient'].append(list(visiting)[0])
    recipesDf = pd.DataFrame.from_dict(recipes)
    df = pd.concat([df, recipesDf])
    
    a=soup2.findAll('div', class_='sr_rows clearfix ')
    count = 0
    for recipeList in a:
        count += 1
        if recipeList.find('img').get('src') != "/css/i/recipe-img-icon.png":
            #allRecipes.append(recipeList.a.get('href'))
            recipes['recipe'].append(recipeList.a.get('href'))
            recipes['searchIngredient'].append(list(visiting)[0])
            recipesDf = pd.DataFrame.from_dict(recipes)
            df = pd.concat([df, recipesDf])
    
    if (count < 28) | (pageNumber > 100):
        return getRecipes(df, veggiesList, 1, visiting = None)
    else:
        return getRecipes(df, veggiesList, pageNumber+1, visiting)
  
df = pd.DataFrame(columns = ('recipe', 'searchIngredient'))
allRecipes = []
initDf = getRecipes(df, ['vegan'], 1)
initDf.to_pickle('recipeList4')

In [ ]:
### Had saved recipes per 
df = pd.DataFrame(columns = ('recipe', 'searchIngredient'))
allRecipes = []
#initDf = getRecipes(df, veggies1, 1)
#initDf.to_pickle('recipeList3')
df1 = pd.read_pickle('/Volumes/LittleOne/Insight/RecipeProject/recipeList')
df2 = pd.read_pickle('/Volumes/LittleOne/Insight/RecipeProject/recipeList2')
df3 = pd.read_pickle('/Volumes/LittleOne/Insight/RecipeProject/recipeList3')
df4 = pd.read_pickle('/Volumes/LittleOne/Insight/RecipeProject/recipeList6')
df = pd.concat([df1, df2, df3, df4]).drop_duplicates('recipe')

In [ ]:
len(df)

In [ ]:
browser = webdriver.Chrome("chromedriver")
rDf = pd.DataFrame(columns = ('id', 'name', 'tags', 'description', 'imageURL', 'rating', 'willPrepareAgainRating',\
                                  'reviewCount', 'numberOfServings', 'activeTime', 'totalTime', 'numberOfCalories',\
                                  'ingredientsList', 'reviewText'))
recipes = defaultdict(list)
recipeIDs = list(set(df.recipe))
oldRecipesDf1 =  pd.read_pickle('/Volumes/LittleOne/Insight/RecipeProject/recipesDb1')
oldRecipesDf2 =  pd.read_pickle('/Volumes/LittleOne/Insight/RecipeProject/recipesDb2')
oldRecipesDf3 =  pd.read_pickle('/Volumes/LittleOne/Insight/RecipeProject/recipesDb3')
oldRecipesDf4 =  pd.read_pickle('/Volumes/LittleOne/Insight/RecipeProject/recipesDb4')
oldRecipesDf5 =  pd.read_pickle('/Volumes/LittleOne/Insight/RecipeProject/recipesDb5')
oldRecipesDf6 =  pd.read_pickle('/Volumes/LittleOne/Insight/RecipeProject/recipesDb6')
oldRecipesDf = pd.concat([oldRecipesDf1, oldRecipesDf2, oldRecipesDf3, oldRecipesDf4, oldRecipesDf5, oldRecipesDf6])
recipeIDs = list(set(df.recipe)-set(oldRecipesDf.id)) 

for recipeID in recipeIDs:
    url = "http://www.epicurious.com"+recipeID
    browser.get(url)
    
    recipePage = browser.page_source
    soup = BeautifulSoup(recipePage, "html.parser")
    if (soup.find(itemprop = "ratingValue") is not None) & (soup.find(itemprop = "keywords") is not None):
        recipes['id'].append(recipeID)
        recipes['name'].append(soup.find(property = "og:title").get('content'))
        if soup.find(itemprop = "keywords") is None:
            recipes['tags'].append(' ')
        else:
            recipes['tags'].append(soup.find(itemprop = "keywords").get('content'))
        if soup.find(property = "og:description") is None:
            recipes['description'].append(' ')
        else:
            recipes['description'].append(soup.find(property = "og:description").get('content'))
        recipes['imageURL'].append(soup.find(property = "og:image").get('content'))
    
    
        recipes['rating'].append(soup.find(itemprop = "ratingValue").get('content'))
        recipes['willPrepareAgainRating'].append(soup.find('div', class_="prepare-again-rating").span.get_text())
        recipes['reviewCount'].append(soup.find('span', class_="reviews-count").get_text())

        if soup.find('div', class_ = "recipe-summary") is None:
            recipes['numberOfServings'].append(' ')
        else:
            recipes['numberOfServings'].append(soup.find('div', class_ = "recipe-summary").dd.get_text())

        if ((soup.find('dd', class_ = "active-time")) is None) |  (soup.find('dd', class_ = "total-time") is None):
            recipes['activeTime'].append(' ')
            recipes['totalTime'].append(' ')
        else:
            recipes['activeTime'].append(soup.find('dd', class_ = "active-time").get_text())
            recipes['totalTime'].append(soup.find('dd', class_ = "total-time").get_text())

        if soup.find(itemprop = "calories") is None:
            recipes['numberOfCalories'].append(' ')
        else:
            recipes['numberOfCalories'].append(soup.find(itemprop = "calories").get_text())

        # Create a list of ingredients
        ingredientsList = [] 
        ingredients = soup.findAll(itemprop = "ingredients")
        for ingredient in ingredients:
            ingredientsList.append(ingredient.get_text())
        recipes['ingredientsList'].append(ingredientsList)

        # Create a list of reviews
        reviewText = []
        reviews = soup.findAll('div', class_ = "review-text")
        for review in reviews:
            reviewText.append(review.p.get_text())
        recipes['reviewText'].append(reviewText)

recipesDf = pd.DataFrame.from_dict(recipes)
recipesDf.to_pickle('recipesDb6')

In [ ]:
recipesDf = pd.DataFrame.from_dict(recipes)
recipesDf.to_pickle('recipesDb6')

In [ ]:
oldRecipesDf1 =  pd.read_pickle('/Volumes/LittleOne/Insight/RecipeProject/recipesDb1')
oldRecipesDf2 =  pd.read_pickle('/Volumes/LittleOne/Insight/RecipeProject/recipesDb2')
oldRecipesDf3 =  pd.read_pickle('/Volumes/LittleOne/Insight/RecipeProject/recipesDb3')
oldRecipesDf4 =  pd.read_pickle('/Volumes/LittleOne/Insight/RecipeProject/recipesDb4')
oldRecipesDf5 =  pd.read_pickle('/Volumes/LittleOne/Insight/RecipeProject/recipesDb5')
oldRecipesDf6 =  pd.read_pickle('/Volumes/LittleOne/Insight/RecipeProject/recipesDb6')
oldRecipesDf = pd.concat([oldRecipesDf1, oldRecipesDf2, oldRecipesDf3, oldRecipesDf4, oldRecipesDf5, oldRecipesDf6])
recipeIDs = list(set(df.recipe)-set(oldRecipesDf.id))
len(recipeIDs), len(oldRecipesDf)

In [ ]:
recipesDf = pd.DataFrame.from_dict(recipes)
recipesDf.to_pickle('recipesDb5')

In [ ]:
recipesDf = pd.DataFrame.from_dict(recipes)
recipesDf.to_pickle('recipesDb2')
oldRecipesDf.to_pickle('recipesDb1')

### get food spoilage dates

In [ ]:
### get food spoilage dates
foodSpoilURL = "http://ohmyveggies.com/how-to-prevent-food-spoilage-with-careful-meal-planning/"
#page = urllib2.urlopen(foodSpoilURL)
hdr = {'User-Agent':'Mozilla/5.0'}
req = urllib2.Request(foodSpoilURL,headers=hdr)
page = urllib2.urlopen(req)
foodSpoilSoup = BeautifulSoup(page, "html.parser")
foodSpoilHtml = foodSpoilSoup.findAll("p")

In [ ]:
foodSpoilsinDate12Days = foodSpoilHtml[7].get_text().split()
foodSpoilsin35Days = foodSpoilHtml[9].get_text().split()
foodSpoilsin67Days = foodSpoilHtml[11].get_text().split()
foodSpoilsGreatWeek = foodSpoilHtml[13].get_text().split()
foodSpoilDates = [foodSpoilsinDate12Days, foodSpoilsin35Days, foodSpoilsin67Days, foodSpoilsGreatWeek]
with open("foodspoils.txt", "wb") as fo:
    pickle.dump(foodSpoilDates, fo)


### Create a SQL database for my data

In [ ]:
dbname = 'recipes_db'
username = 'nemo'
pswd = 'eatsalot'

engine = create_engine('postgresql://%s:%s@localhost/%s'%(username,pswd,dbname))
print engine.url

In [ ]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))


In [ ]:
reviewText


In [ ]:
len(letters)

### Parse a list of vegetables!

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
def lemmatizePhrase(phrase):
    """ 
    Lemmatize each word in a pharse
    """
    words = phrase.lower().split()
    return ' '.join(WordNetLemmatizer().lemmatize(word) for word in words)

ingredientsURL = "https://en.m.wikibooks.org/wiki/Cookbook:Ingredients"
page = urllib2.urlopen(ingredientsURL)
ingredientsSoup = BeautifulSoup(page, "html.parser")
ingredientsHtml = ingredientsSoup.findAll("li")
ingredients = []

for ingredient in ingredientsHtml:
    if ingredient.a is not None:
        ingredients.append(lemmatizePhrase(ingredient.a.get_text()))

ingredients = list(set(ingredients))
ingredients.remove('cc by-sa 3.0')
ingredients.remove('')
ingredients.remove('edit')
ingredients.remove('desktop')
ingredients.remove('privacy')
ingredients.remove('soda, baking')
veggies = []
for veggie in veggieList:
    ingredients.append(lemmatizePhrase(veggie.lower()))
    veggies.append(lemmatizePhrase(veggie.lower()))
ingredients = list(set(ingredients))
with open("ingredients.txt", "wb") as fo:
    pickle.dump(ingredients, fo)
with open("veggies.txt", "wb") as fo:
    pickle.dump(veggies, fo)

### Parse a list of food adjectives

In [ ]:
foodAdjectivesURL = "https://foxhugh.com/word-lists/list-of-food-adjectives/"
page = urllib2.urlopen(foodAdjectivesURL)
foodAdjectivesSoup = BeautifulSoup(page, "html.parser")
foodAdjectivesHtml = foodAdjectivesSoup.findAll("p")
adjectives = []
for adjective in foodAdjectivesHtml:
    adjectives.append(lemmatizePhrase(adjective.get_text()))
adjectives = adjectives[1:302]
with open("adjectives.txt", "wb") as fo:
    pickle.dump(adjectives, fo)

In [ ]:
for veggie in veggieList:
    ingredients.append(lemmatizePhrase(veggie.lower()))

In [ ]:
len(ingredients)

In [ ]:
len(list(set(ingredients)))